In [1]:
# 导入pytorch，需要在对应的conda环境安装torch，本conda环境为studyDeeplearningProject，使用语句“pip install torch
import torch

In [3]:
# 张量表示一个数值组成的数组，这个数组可能有多个维度
x = torch.arange(12) #表示生成一个一维从0-11的一个张量
x
# 输出结果为：tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [4]:
#x.shape可以查看元素的形状
x.shape

torch.Size([12])

In [5]:
#x.reshape可以改变张量的形状
x.reshape(3,4)

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])

In [6]:
torch.zeros(2,3,4)#生成元素全0的张量，后面参数指定张量形状
torch.ones(1,2,3)#生成元素全为1的张量，后面参数指定张量形状

tensor([[[1., 1., 1.],
         [1., 1., 1.]]])

In [8]:
#也可以提供指定数值的列表创建向量
y = torch.tensor([1,2,3,4,5,6])
y

tensor([1, 2, 3, 4, 5, 6])

In [12]:
#张量的加减乘除基本运算都是按元素运算
#也可以将多个张量连接在一起。使用cat函数，dim=一个数，则将两个张量按照第几维（也就是往里面数这个数+1个括号）堆叠起来，例如dim=0，则往里面数一个括号，则剩下的内容堆起来
x = torch.arange(12,dtype=torch.float32).reshape((3,4))
y = torch.tensor([[5,6,7,8],[9,0,1,2],[3,4,5,6]])
torch.cat((x,y),dim=0),torch.cat((x,y),dim=1)

(tensor([[ 0.,  1.,  2.,  3.],
         [ 4.,  5.,  6.,  7.],
         [ 8.,  9., 10., 11.],
         [ 5.,  6.,  7.,  8.],
         [ 9.,  0.,  1.,  2.],
         [ 3.,  4.,  5.,  6.]]),
 tensor([[ 0.,  1.,  2.,  3.,  5.,  6.,  7.,  8.],
         [ 4.,  5.,  6.,  7.,  9.,  0.,  1.,  2.],
         [ 8.,  9., 10., 11.,  3.,  4.,  5.,  6.]]))

In [13]:
x.sum()

tensor(66.)

In [14]:
#广播机制，两个张量必须在维度相同，否则无法广播，广播是将一个小的数量向大的扩大，例如下面一个3*1的张量a，一个1*2的张量b，则a会在列上扩大成2，b会在行上面扩大成3，都变成3*2的张量（扩大的过程是将元素复制），再进行运算。
a = torch.arange(3).reshape((3,1))
b = torch.arange(2).reshape((1,2))
a,b

(tensor([[0],
         [1],
         [2]]),
 tensor([[0, 1]]))

In [15]:
a+b

tensor([[0, 1],
        [1, 2],
        [2, 3]])

In [ ]:
#张量的索引从0开始，-1表示最后一个

In [17]:
#python中的=赋值操作会重新分配内存，底层的存储地址会发生改变
Y = torch.zeros((1,2))
before = id(Y)
Y = Y+a
id(Y) == before

False

In [27]:
#执行原地操作，即不重新分配内存,使用z[:]，考虑python的内存重新分配机制，在有些时候需要考虑原地操作，防止重新分配内存消耗时间
z = torch.ones((1,2))
x = torch.zeros((1,2))
before = id(z)
z[:] = x + z
before == id(z)

True

In [29]:
#使用自加,自乘等操作也是原地操作
print("before id :",id(z))
z += 1
print("after id:",id(z))

before id : 2882351805392
after id: 2882351805392


In [30]:
# 将大小为1的张量转换成python的标量
a = torch.tensor(3.4)
a, a.item(), float(a)

(tensor(3.4000), 3.4000000953674316, 3.4000000953674316)

In [24]:
#数据预处理，深度学习的基本操作
#创建一个人工数据集
import os  #os模块，提供丰富的方法处理文件和目录
os.makedirs(os.path.join('..','data'),exist_ok=True) #makedirs,递归生成文件夹，path.join实现路径拼接
data_file = os.path.join('..','data','house_tiny.csv')
with open(data_file,'w') as f:
    f.write('NumRooms,Alley,Price\n')
    f.write('NA,Pave,127500\n')
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write ('NA,NA,140000\n')

In [25]:
import pandas as pd
#pandas提供读取csv文件的函数
data = pd.read_csv(data_file,encoding="gbk")
data

,NumRooms,Alley,Price
0,NaN,Pave,127500
1,2.0,NaN,106000
2,4.0,NaN,178100
3,NaN,NaN,140000


In [26]:
#处理缺失数据，常用方法是插值和删除，这里考虑插值
inputs, outputs = data.iloc[:, 0:2],data.iloc[:, 2]#将输入和输出分开，iloc用于分割表格
inputs,outputs

(   NumRooms Alley
 0       NaN  Pave
 1       2.0   NaN
 2       4.0   NaN
 3       NaN   NaN,
 0    127500
 1    106000
 2    178100
 3    140000
 Name: Price, dtype: int64)

In [32]:
#用存在的元素的平均值填充
inputs = inputs.fillna(inputs.mean(numeric_only=True))
inputs
##原直接用inputs.mean()此处出现报错信息：TypeError: can only concatenate(把什么连接起来) str (not "int") to str
#原因是inputs.mean() 返回包括数值型和非数值型列的均值。NaN在mean中不能当被处理
#改错方式为：使用inputs.mean(numeric_only=True)仅返回数值型列的均值

,NumRooms,Alley
0,3.0,Pave
1,2.0,NaN
2,4.0,NaN
3,3.0,NaN
